In [375]:
from collections import Counter
import random
import string
import Levenshtein

In [376]:
PRINT_LEN = 50

In [377]:
def get_accuracy(pred, label):
    corrects = 0
    for i in range(len(pred)):
        if pred[i] == label[i]:
            corrects += 1
    acc = corrects / len(pred)
    distance = Levenshtein.distance(pred, label)
    print(f'Accuracy: {acc:.3f}  Levenstain distance: {distance}')
    return acc, distance

In [378]:
data_path = 'texts/WarAndPeace.txt'
with open(data_path, 'r') as f:
    corpus = f.read()
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
corpus = corpus.lower().replace('\n', ' ').replace('\t', ' ')
corpus = ''.join([x for x in corpus if x in alphabet])
corpus = ' '.join(corpus.split())
corpus[:PRINT_LEN]


'война и мир самый известный роман льва николаевича'

### 1. Базовый частотный метод

In [379]:
unigrams = Counter(corpus).most_common()
unigrams[:5]

[(' ', 103408), ('о', 61282), ('а', 45209), ('е', 42519), ('и', 35838)]

In [380]:
test = corpus[:10000]

unique_chars = list(set(test))
unique_chars_shuffled = unique_chars.copy()
random.Random(42).shuffle(unique_chars_shuffled)

encoder_dict = dict(zip(unique_chars, unique_chars_shuffled))
trans_table = str.maketrans(encoder_dict)

test_encoded = test.translate(trans_table)
encoded_unigrams = Counter(test_encoded).most_common()
print('true:   ', test[:PRINT_LEN])
print('encoded:', test_encoded[:PRINT_LEN])

true:    война и мир самый известный роман льва николаевича
encoded: фжкдмлрлырхльмысклрпфньбдсклхжымдлъщфмлдрчжъмнфрём


In [381]:
sorted(list(set(corpus)))

[' ',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'ё']

In [382]:
print('true:    ', test[:PRINT_LEN])
print('encoded: ', test_encoded[:PRINT_LEN])
for i in range(40):
    decoder_dict = dict(zip([x[0] for x in encoded_unigrams[:i + 1]], [x[0] for x in unigrams[:i + 1]]))
    trans_table = str.maketrans(decoder_dict)
    test_decoded = test_encoded.translate(trans_table)
    print(f'top {i + 1:2} :', test_decoded[:PRINT_LEN], end='   ')
    get_accuracy(test, test_decoded);

true:     война и мир самый известный роман льва николаевича
encoded:  фжкдмлрлырхльмысклрпфньбдсклхжымдлъщфмлдрчжъмнфрём
top  1 : фжкдм р ырх ьмыск рпфньбдск хжымд ъщфм дрчжъмнфрём   Accuracy: 0.162  Levenstain distance: 8342
top  2 : фокдм р ырх ьмыск рпфньбдск хоымд ъщфм дрчоъмнфрём   Accuracy: 0.251  Levenstain distance: 7463
top  3 : фокдм р ырх ьмыск рпфаьбдск хоымд ъщфм дрчоъмафрём   Accuracy: 0.251  Levenstain distance: 7469
top  4 : фокде р ырх ьеыск рпфаьбдск хоыед ъщфе дрчоъеафрёе   Accuracy: 0.251  Levenstain distance: 7472
top  5 : фокие р ырх ьеыск рпфаьбиск хоыеи ъщфе ирчоъеафрёе   Accuracy: 0.251  Levenstain distance: 7474
top  6 : фокие н ынх ьеыск нпфаьбиск хоыеи ъщфе инчоъеафнёе   Accuracy: 0.251  Levenstain distance: 7480
top  7 : фокие н ынх ьеыск нпфаьтиск хоыеи ъщфе инчоъеафнёе   Accuracy: 0.301  Levenstain distance: 6981
top  8 : сокие н ынх ьеыск нпсаьтиск хоыеи ъщсе инчоъеаснёе   Accuracy: 0.301  Levenstain distance: 6980
top  9 : сокие н ынх леыск нпсалтиск х

In [265]:
decoder_dict

{'л': ' ',
 'ж': 'о',
 'н': 'а',
 'м': 'е',
 'д': 'и',
 'р': 'н',
 'б': 'т',
 'ф': 'с',
 'ь': 'л',
 'х': 'в',
 'ъ': 'р',
 'ч': 'к',
 'ы': 'д',
 'з': 'м',
 'т': 'у',
 'ю': 'п',
 'ц': 'я',
 'п': 'г',
 'с': 'ь',
 'щ': 'ы',
 'ё': 'з',
 'э': 'б',
 'в': 'ч',
 'к': 'й',
 'ш': 'ж',
 'о': 'ш',
 'й': 'х',
 'е': 'ю',
 'я': 'ц',
 'и': 'э',
 ' ': 'щ',
 'г': 'ф',
 'у': 'ё',
 'а': 'ъ'}

### 2. Биграммный частотный метод

In [350]:
bigrams = Counter(corpus[x:x+2] for x in range(len(corpus) - 1)).most_common()
print(f'bigrams: {len(bigrams)}')
bigrams[:5]

bigrams: 796


[('о ', 13316), ('и ', 11398), ('а ', 10596), ('е ', 10040), (' с', 9863)]

In [267]:
unique_chars = Counter(corpus[x:x+2] for x in range(len(corpus) - 1)).most_common() 
unique_chars = [x[0] for x in unique_chars]
unique_chars_shuffled = unique_chars.copy()
random.Random(42).shuffle(unique_chars_shuffled)

encoder_dict = dict(zip(unique_chars, unique_chars_shuffled))

test_encoded = ''.join([encoder_dict[test[i:i+2]] for i in range(0, len(test), 2)])
encoded_bigrams = Counter(test_encoded[x:x+2] for x in range(len(test_encoded) - 1)).most_common() 
print(test[:100])
print(test_encoded[:100])

война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя
вчькдмрзтоьснбуепгсебайшктпгядъессгшруепмзршюсэдеуёмршйшё цесржбвзсрйллбяладзцдшядсебакдвзлбнэнбстяг


In [268]:
print('true:    ', test[:PRINT_LEN])
print('encoded: ', test_encoded[:PRINT_LEN])
for i in range(20):
    decoder_dict = dict(zip([x[0] for x in encoded_bigrams[:i + 1]], [x[0] for x in bigrams[:i + 1]]))
    test_decoded = ''.join([decoder_dict[test_encoded[i:i+2]] if test_encoded[i:i+2] in decoder_dict else test_encoded[i:i+2] for i in range(0, len(test_encoded), 2)])
    print(f'top {i + 1:2} :', test_decoded[:PRINT_LEN], end='   ')
    get_accuracy(test, test_decoded);

true:     война и мир самый известный роман льва николаевича
encoded:  вчькдмрзтоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу
top  1 : вчькдмо тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.047  Levenstain distance: 8815
top  2 : вчькдмо тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.057  Levenstain distance: 8761
top  3 : вчькдмо тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.066  Levenstain distance: 8727
top  4 : вчьке о тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.076  Levenstain distance: 8676
top  5 : вчьке о тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.084  Levenstain distance: 8655
top  6 : вчьке о тоьснбуепгсебайшктпгядъессгшру пмзршюсэдеу   Accuracy: 0.092  Levenstain distance: 8631
top  7 : вчьке о тоьснбуепгсебайшктпгядъессгшру пмзршюсэдеу   Accuracy: 0.092  Levenstain distance: 8646
top  8 : вчьке о тоьснбуепгсебайшктпгядъессгшру пмзршюсэдеу   Accuracy: 0.092  Levenstain distance: 8621
top  9 : вчьке о тоьснбуепгсебайшктпгя

In [269]:
decoder_dict

{'рз': 'о ',
 'лб': 'и ',
 'це': 'а ',
 'дм': 'е ',
 'ео': ' с',
 'еп': ' п',
 ' о': ' в',
 'ев': ' н',
 'дш': 'то',
 'ач': ' о',
 'т ': 'я ',
 'вш': ' к',
 'зж': ' и',
 'ые': 'ст',
 'шр': 'ь ',
 'пг': 'на',
 'йш': 'го',
 'шв': 'ал',
 'сш': 'не',
 'дк': 'по'}

### 3. MCMC-сэмплированиe

In [270]:
import numpy as np
import pandas as pd

In [271]:
# corpus = 'abcadaddegfc  abcada cbaddcfffff'

In [272]:
bigrams = Counter(corpus[x:x+2] for x in range(len(corpus) - 1)).most_common()
bigrams = [x[0] for x in bigrams]
bigrams[:10], len(bigrams)

(['о ', 'и ', 'а ', 'е ', ' с', ' п', ' в', ' н', 'то', ' о'], 796)

In [273]:
token2id = {token: i for i, token in enumerate(bigrams)}
id2token = {i: token for token, i in token2id.items()}
token2id;

static_distr = np.zeros((len(bigrams), len(bigrams)))
static_distr.shape

(796, 796)

In [274]:
static_distr = np.zeros((len(bigrams), len(bigrams)))
for i in range(len(corpus) - 3):
    cur_token = token2id[corpus[i:i+2]]
    next_token = token2id[corpus[i+2:i+4]]
    static_distr[cur_token][next_token] += 1

sort_keys = static_distr.max(axis=1).argsort()[::-1]
static_distr_sorted = static_distr[sort_keys].T[sort_keys].T
bigrams_sorted = [bigrams[i] for i in sort_keys]

In [337]:
from scipy.special import softmax

In [336]:
static_distr

array([[ 30., 501.,  32., ...,   0.,   0.,   0.],
       [ 58., 478.,  49., ...,   0.,   0.,   0.],
       [ 20., 581.,  55., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [275]:
sort_keys

array([ 56,   7,   9,  94,  91,  63,  27,  11,  26,   5, 125,  61,  75,
        62,   4, 132,  76,  23, 181,  16,  37,  22,  17,  33, 127,   0,
         6,  12,  36, 112, 166,  95,   1,  29,  51,  78, 118,   2, 138,
        35,  54,  68, 143,  73,  39,  96,   3,  65,  38, 135,   8,  84,
        52,  80,  44,  42,  30,  34,  60, 208,  88,  31,  55, 242,  13,
       108, 204,  71,  20, 106,  10,  57, 243,  24,  79, 103,  41,  46,
        15, 155, 122,  32,  48, 109, 102,  14, 241,  49, 297,  59,  43,
       226,  18,  85, 113, 151,  47,  53, 182, 161, 124,  50, 210,  28,
       133, 285,  40, 251,  19, 170, 137, 178,  70,  77, 160, 107,  21,
       194, 111, 322, 150, 139, 171, 215, 319, 244,  25,  97, 257,  67,
       159, 213, 288, 175, 219, 129, 205, 110, 245, 134, 141, 189, 301,
       101, 298, 248, 146, 294, 234, 173, 114,  86,  89, 306,  83, 202,
       358,  90,  72, 126, 123, 121, 218, 116, 321, 186, 147, 119, 167,
       174, 217,  82, 158, 104, 130,  66, 149, 231, 225,  69, 30

In [276]:
static_distr.max(axis=1)

array([6.630e+02, 6.170e+02, 5.810e+02, 5.160e+02, 9.400e+02, 1.198e+03,
       6.570e+02, 1.973e+03, 5.000e+02, 1.807e+03, 3.980e+02, 1.245e+03,
       6.530e+02, 4.160e+02, 3.470e+02, 3.690e+02, 8.210e+02, 7.330e+02,
       3.220e+02, 2.800e+02, 4.010e+02, 2.610e+02, 7.950e+02, 8.350e+02,
       3.880e+02, 2.460e+02, 1.208e+03, 1.247e+03, 2.930e+02, 6.110e+02,
       4.700e+02, 4.330e+02, 3.570e+02, 7.110e+02, 4.620e+02, 5.770e+02,
       6.400e+02, 8.020e+02, 5.080e+02, 5.260e+02, 2.840e+02, 3.750e+02,
       4.720e+02, 3.260e+02, 4.730e+02, 1.750e+02, 3.730e+02, 3.090e+02,
       3.530e+02, 3.450e+02, 2.990e+02, 6.110e+02, 4.830e+02, 3.080e+02,
       5.590e+02, 4.270e+02, 2.005e+03, 3.950e+02, 1.710e+02, 3.310e+02,
       4.590e+02, 9.820e+02, 9.630e+02, 1.354e+03, 1.180e+02, 5.150e+02,
       1.910e+02, 2.410e+02, 5.540e+02, 1.850e+02, 2.720e+02, 4.030e+02,
       2.070e+02, 5.290e+02, 1.210e+02, 9.690e+02, 8.490e+02, 2.720e+02,
       6.040e+02, 3.810e+02, 4.780e+02, 1.830e+02, 

In [277]:
bigrams

['о ',
 'и ',
 'а ',
 'е ',
 ' с',
 ' п',
 ' в',
 ' н',
 'то',
 ' о',
 'я ',
 ' к',
 ' и',
 'ст',
 'ь ',
 'на',
 'го',
 'ал',
 'не',
 'по',
 'но',
 'ра',
 'ко',
 'ов',
 'л ',
 'й ',
 'ка',
 ' б',
 'м ',
 'во',
 ' д',
 'ро',
 'ер',
 ' т',
 'ол',
 'ен',
 'он',
 'от',
 'ни',
 'ос',
 'у ',
 'ла',
 'ли',
 'ло',
 'ор',
 'в ',
 'ел',
 'ре',
 'пр',
 'ва',
 ' м',
 'за',
 'та',
 'ри',
 ' г',
 'ан',
 ' ч',
 'ил',
 'к ',
 'ть',
 'ом',
 'ат',
 'ск',
 'ог',
 'т ',
 'те',
 'н ',
 'од',
 ' р',
 'ы ',
 'ак',
 'де',
 'ве',
 ' з',
 'да',
 'аз',
 ' е',
 ' у',
 'ем',
 'ас',
 'же',
 'ле',
 'ой',
 'ет',
 'бы',
 'мо',
 'до',
 'ми',
 'ль',
 'ес',
 'ав',
 'чт',
 'ти',
 'ю ',
 'ег',
 'ня',
 'ит',
 'сь',
 'ви',
 'ин',
 'х ',
 'ру',
 'об',
 'ны',
 'ся',
 'с ',
 'вс',
 'че',
 ' а',
 'нн',
 'им',
 'тр',
 'ис',
 'ая',
 'ма',
 'ди',
 'ме',
 'ар',
 'ам',
 'се',
 'вы',
 'ну',
 ' л',
 'ив',
 'ки',
 ' э',
 'со',
 'пе',
 'ед',
 'ие',
 'сл',
 'ей',
 'кн',
 'хо',
 'бо',
 'ыл',
 'му',
 'ше',
 'эт',
 'ча',
 'ад',
 'тв',
 'ое',

In [278]:
bigrams_sorted

[' ч',
 ' н',
 ' о',
 'ег',
 'чт',
 'ог',
 ' б',
 ' к',
 'ка',
 ' п',
 ' э',
 'ат',
 'аз',
 'ск',
 ' с',
 'кн',
 ' е',
 'ов',
 'лс',
 'го',
 'от',
 'ко',
 'ал',
 ' т',
 'пе',
 'о ',
 ' в',
 ' и',
 'он',
 'ис',
 'яз',
 'ня',
 'и ',
 'во',
 'за',
 'ем',
 'ам',
 'а ',
 'эт',
 'ен',
 ' г',
 ' р',
 'др',
 ' з',
 'ос',
 'ит',
 'е ',
 'те',
 'ни',
 'ыл',
 'то',
 'бы',
 'та',
 'же',
 'ор',
 'ли',
 ' д',
 'ол',
 'ом',
 'ьк',
 'ль',
 'ро',
 'ан',
 'яс',
 'ст',
 ' а',
 'нд',
 'де',
 'но',
 'вс',
 'я ',
 'ил',
 'гд',
 'л ',
 'ас',
 'ны',
 'ла',
 'ел',
 'на',
 ' ж',
 ' л',
 'ер',
 'пр',
 'нн',
 'об',
 'ь ',
 'ьс',
 'ва',
 'пь',
 'ть',
 'ло',
 'см',
 'не',
 'мо',
 'ая',
 'св',
 'ре',
 'ри',
 'гр',
 'ду',
 'ки',
 ' м',
 'зь',
 'м ',
 'хо',
 'ещ',
 'у ',
 'тс',
 'по',
 'ул',
 'ше',
 'ят',
 'ак',
 ' у',
 'гл',
 'че',
 'ра',
 'аж',
 'тр',
 'оф',
 'кр',
 'ча',
 'ым',
 'еб',
 'фр',
 'уз',
 'й ',
 'сь',
 ' ф',
 'од',
 'дн',
 'ыв',
 'фи',
 'иц',
 'ьн',
 'ие',
 'мн',
 'им',
 'ае',
 'бо',
 'тв',
 'уж',
 'нц',

In [287]:
diag_max = static_distr.diagonal().argmax()
static_distr[diag_max, diag_max], bigrams[diag_max], bigrams[diag_max] * 2 in corpus

(478.0, 'и ', True)

In [289]:
diag_max = static_distr_sorted.diagonal().argmax()
static_distr_sorted[diag_max, diag_max], bigrams_sorted[diag_max], bigrams_sorted[diag_max] * 2 in corpus

(478.0, 'и ', True)

In [290]:
true_distr_id = {}
true_distr_token = {}
for i, id in enumerate(static_distr_sorted.argmax(axis=1)):
    true_distr_id[i] = id
    true_distr_token[id2token[i]] = id2token[id]

In [291]:
test_encoded

'вчькдмрзтоьснбуепгсебайшктпгядъессгшруепмзршюсэдеуёмршйшё цесржбвзсрйллбяладзцдшядсебакдвзлбнэнбстягт шшжаюсифнкпьащипутцетояднр гзжбх оуябъсаппрблюфкдавзмм ощнсшыснгдмбакж мдшяшшаипояцетидммпцеш зйт йрсевз овшя ибрзцеткуврзрзжмйшрзцерыашйшба озкр ымющлбцепэяфлбрзвгвсшллбцевчьклбрзтокедшр лчпгачвбяидхшашнвшеодожчвбмпрдйшвчвбяд ярбеозддмвгсршвктеошвовлгтмдизжбхафтуюхдпиошнмклмрзрзкдл тоеовшзйушдпсаачлчъ идвмюшйшвчожубршйлаяжуъ ащшшшлрзипфмойдблчъ бажмсшеовшящтилбжкмгьбкт тдур р убящсдиукронба одшг р уонилбсгиувзйлнщюнбрёмршйшмлеуйшыеьдэхалгчзыл т сшдкеуйшыешшг рэывюнепмзршюсэдшиачкиачпгвчькдмрзтоьсачвбеуйшыеьдэхалжэдшшам л дккрпгбаищпыдпрцепипояцебунцыеянгтдмрзцкббдмйшящрзпиубршоблбсржбвгя йшрэтоуябзбърбубти жыжстепг осеозыдшкеюхпчтурмсфм ьбеосдечлбпившсрашстгчачожепзыеомлевжэпэрзлчжэрддштдвчьбствш члбшврпзчл еойцзк йсаеяеойцожжжддлюачб  тзттцыдйшдмпчяфцедкбачегчачт сстлеягыимифдкрурв цгшугеплбяилоеосду еплбяладафмлеосду еплбкрпгбаищпыдпрцдадхеосдб вчыдстепипмшсшш шеохстс яляфйшхеазлбд

In [304]:
encoded_bigrams = Counter(test_encoded[x:x+2] for x in range(len(test_encoded) - 1)).most_common()
encoded_bigrams = [x[0] for x in encoded_bigrams]
len(encoded_bigrams)

961

In [306]:
encoded_token2id = {token: i for i, token in enumerate(encoded_bigrams)}
encoded_id2token = {i: token for token, i in encoded_token2id.items()}

In [307]:
encoded_static_distr = np.zeros((len(encoded_bigrams), len(encoded_bigrams)))
encoded_static_distr.shape

(961, 961)

In [313]:
for i in range(len(test_encoded) - 3):
    cur_token = encoded_token2id[test_encoded[i:i+2]]
    next_token = encoded_token2id[test_encoded[i+2:i+4]]
    encoded_static_distr[cur_token][next_token] += 1

encoded_sort_keys = encoded_static_distr.max(axis=1).argsort()[::-1]
encoded_static_distr_sorted = encoded_static_distr[encoded_sort_keys].T[encoded_sort_keys].T
encoded_bigrams_sorted = [encoded_bigrams[i] for i in encoded_sort_keys]

In [314]:
encoded_static_distr_sorted.shape

(961, 961)

In [315]:
pred_distr_id = {}
pred_distr_token = {}
for i, id in enumerate(encoded_static_distr_sorted.argmax(axis=1)):
    pred_distr_id[i] = id
    pred_distr_token[encoded_id2token[i]] = encoded_id2token[id]

In [329]:
len(list(true_distr_token.items())), len(list(pred_distr_token.items()))

(796, 961)

In [330]:
decoder_dict = {}
for i in range(len(true_distr_token)):
    decoder_dict[list(pred_distr_token.items())[i][0]] = list(true_distr_token.items())[i][0]

In [333]:
print('true:    ', test[:PRINT_LEN])
print('encoded: ', test_encoded[:PRINT_LEN])
decoder_dict = {}
for i in range(20):
    decoder_dict[list(pred_distr_token.items())[i][0]] = list(true_distr_token.items())[i][0]
    test_decoded = ''.join([decoder_dict[test_encoded[i:i+2]] if test_encoded[i:i+2] in decoder_dict else test_encoded[i:i+2] for i in range(0, len(test_encoded), 2)])
    print(f'top {i + 1:2} :', test_decoded[:PRINT_LEN], end='   ')
    get_accuracy(test, test_decoded);

true:     война и мир самый известный роман льва николаевича
encoded:  вчькдмрзтоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу
top  1 : вчькдмо тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.047  Levenstain distance: 8815
top  2 : вчькдмо тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.057  Levenstain distance: 8761
top  3 : вчькдмо тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.066  Levenstain distance: 8727
top  4 : вчьке о тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.076  Levenstain distance: 8676
top  5 : вчьке о тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.084  Levenstain distance: 8655
top  6 : вчьке о тоьснбуепгсебайшктпгядъессгшру пмзршюсэдеу   Accuracy: 0.092  Levenstain distance: 8631
top  7 : вчьке о тоьснбуепгсебайшктпгядъессгшру пмзршюсэдеу   Accuracy: 0.092  Levenstain distance: 8646
top  8 : вчьке о тоьснбуепгсебайшктпгядъессгшру пмзршюсэдеу   Accuracy: 0.092  Levenstain distance: 8621
top  9 : вчьке о тоьснбуепгсебайшктпгя

In [405]:
vocab = sorted(list(set(corpus)))
token2id = {token: id for id, token in enumerate(vocab)}

transition_matrix = np.zeros((len(vocab), len(vocab))).astype(int)
for i in range(len(corpus) - 1):
    cur_token = token2id[corpus[i:i+1]]
    next_token = token2id[corpus[i+1:i+2]]
    transition_matrix[cur_token][next_token] += 1
transition_matrix


array([[    0,  1881,  4700, ...,    41,  1005,     1],
       [10596,    10,   425, ...,   497,  1780,     0],
       [  154,   733,    16, ...,     4,   298,     0],
       ...,
       [ 2140,     0,   197, ...,    23,     0,     0],
       [ 7096,     1,    21, ...,    55,   128,     0],
       [  402,     0,     0, ...,     0,     0,     0]])

In [409]:
n_grams = vocab.copy()
def change_text(text):
    return text.replace(random.choice(n_grams)[0], random.choice(n_grams)[0])

def get_score(text, matrix):
    score = 0
    for i in range(len(text)-1):
        # if text[i:i+2] in token2id and text[i+:i+4] in token2id:
        cur_token = token2id[text[i]]
        next_token = token2id[text[i+1]]
        score += matrix[cur_token, next_token]
    return score

print(test[:100])
num_iters = 20000
cur_text = test_encoded
matrix = transition_matrix / transition_matrix.sum(axis=1)
matrix = np.log(matrix + 1)
best_score = get_score(cur_text, matrix)
for i in range(num_iters):
    new_text = change_text(cur_text)
    cur_score = get_score(new_text, matrix)
    if cur_score >= best_score:
        cur_text = new_text
        best_score = cur_score
    elif np.log(random.random()) < best_score - cur_score:
        cur_text = new_text
    if i % 2000 == 0:
        print(i, cur_text[:100], cur_score)

# cur_text


война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя
0 фжкдмлрлырхльмысклрпфньбдсклхжымдлъщфмлдрчжъмнфрёмлбжъьбжэжлчмчлдрчмчжнлзхрэжнлцхжрпфнзндрнлцрьмбнъю 159.76579055371263
2000 оооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооооо 19.72334665674684
4000 дддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддддд 7.929179663392021
6000 ыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыыы 0.0
8000 юююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююю 65.5861470253112
10000 юююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююююю 65.5861470253112
12000 шшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшшш 5.8915841830230296
14000 фффффффффффффффффффффффффффффффффффффффф

In [388]:
(np.log(transition_matrix+1) / np.linalg.norm(np.log(transition_matrix+1), axis=1))

array([[0.06525208, 0.12012715, 0.07054375, ..., 0.        , 0.        ,
        0.        ],
       [0.07736454, 0.11923412, 0.07885053, ..., 0.        , 0.        ,
        0.        ],
       [0.05790418, 0.12294156, 0.08111335, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [335]:
random.random()

0.5052176639582108